In [1]:
import pandas as pd
import requests
from datetime import datetime
from sqlalchemy import create_engine
import pickle
import os
uri = os.environ.get('URI')

In [4]:
# Load Files
forecaster_rf = pickle.load(open('model.pickle', 'rb'))
last_training_date = pickle.load(open('last_training_date.pickle', 'rb'))
last_training_date = datetime.strptime(last_training_date, '%Y-%m-%d %H:%M:%S')

In [5]:
url = 'https://api.blockchain.info/charts/transactions-per-second?timespan=all&sampled=false&metadata=false&cors=true&format=json'
resp = requests.get(url)
data = pd.DataFrame(resp.json()['values'])

# I correct the date
data['x'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in data['x']]
data['x'] = pd.to_datetime(data['x'])

# I read the last prediction
engine = create_engine(uri)
query = engine.execute('SELECT MAX(prediction_date) FROM predictions;')
last_prediction_date= query.fetchall()[0][0]
query.close()

In [6]:
last_prediction_date

datetime.datetime(2024, 4, 8, 0, 0)

In [7]:
# If there is no last date in the databse or training > database, I read the last date from training

num_predictions = 168

# if statements dostarczone przez neptune.ai chyba nie są poprawne, trzeba zmienić 
# na odpowiednie !!!

if  (last_prediction_date is None) or (last_prediction_date > last_training_date):

    print("OPTION 1")

    # As there is no predictions, I make the predictions
    predictions = forecaster_rf.predict(num_predictions)

    fechas = pd.date_range(
        start = last_training_date.strftime('%Y-%m-%d %H:%M:%S'),
        periods = num_predictions,
        freq = '1H'
        )

# sytuacje gdzie już używaliśmy naszego wytrenowanego modelu to stworzenia jakiejś predykcji
elif last_prediction_date > last_training_date:

    print("OPTION 2")

    # In this case, we must take into account the differences between the last forecast date and add the difference to the number of days to extract.
    dif_seg= last_prediction_date - last_training_date
    hours_extract = num_predictions + dif_seg.seconds//3600
    predictions = forecaster_rf.predict(num_predictions)
    # I get the last predictions
    predictions = predictions[-num_predictions:]

    fechas = pd.date_range(
        start = last_prediction_date.strftime('%Y-%m-%d %H:%M:%S'),
        periods = num_predictions,
        freq = '1H'
        )
else:

    print("OPTION 3")

    # If last training > last predictions
    predictions = forecaster_rf.predict(num_predictions)

    fechas = pd.date_range(
        start = last_training_date.strftime('%Y-%m-%d %H:%M:%S'),
        periods = num_predictions,
        freq = '1H'
        )
    

fechas

OPTION 3


DatetimeIndex(['2024-04-11 10:00:00', '2024-04-11 11:00:00',
               '2024-04-11 12:00:00', '2024-04-11 13:00:00',
               '2024-04-11 14:00:00', '2024-04-11 15:00:00',
               '2024-04-11 16:00:00', '2024-04-11 17:00:00',
               '2024-04-11 18:00:00', '2024-04-11 19:00:00',
               ...
               '2024-04-18 00:00:00', '2024-04-18 01:00:00',
               '2024-04-18 02:00:00', '2024-04-18 03:00:00',
               '2024-04-18 04:00:00', '2024-04-18 05:00:00',
               '2024-04-18 06:00:00', '2024-04-18 07:00:00',
               '2024-04-18 08:00:00', '2024-04-18 09:00:00'],
              dtype='datetime64[ns]', length=168, freq='H')

In [9]:
upload_data = list(zip([
datetime.now().strftime('%Y-%m-%d %H:%M:%S')] * num_predictions,
[fecha.strftime('%Y-%m-%d %H:%M:%S') for fecha in fechas ],
    predictions
))


upload_data[:10]


[('2024-04-11 14:25:05', '2024-04-11 10:00:00', 12.032578698153703),
 ('2024-04-11 14:25:05', '2024-04-11 11:00:00', 21.26050244135612),
 ('2024-04-11 14:25:05', '2024-04-11 12:00:00', 13.40114460724461),
 ('2024-04-11 14:25:05', '2024-04-11 13:00:00', 20.588279860826205),
 ('2024-04-11 14:25:05', '2024-04-11 14:00:00', 14.005684643572804),
 ('2024-04-11 14:25:05', '2024-04-11 15:00:00', 19.97536599566418),
 ('2024-04-11 14:25:05', '2024-04-11 16:00:00', 14.453226640464795),
 ('2024-04-11 14:25:05', '2024-04-11 17:00:00', 20.68740810755512),
 ('2024-04-11 14:25:05', '2024-04-11 18:00:00', 14.319241527916523),
 ('2024-04-11 14:25:05', '2024-04-11 19:00:00', 20.81007113720903)]